### Heatmap of the top fGWAS results

In [21]:
setwd("~/Family1070/private_output/Fgwas/")

In [22]:
library(plotrix)
suppressPackageStartupMessages(library(gplots))

In [3]:
res = read.table('~/Family1070/private_output/fgwas_analysis/PR_interval/pipeline_out/Final_Model/Final_Model.bfs', header=T)
seg = read.table('~/Family1070/private_output/fgwas_analysis/PR_interval/pipeline_out/Final_Model/Final_Model.segbfs', header=T)
res = merge(res,seg[,c('chunk', 'PPA')], by="chunk" )
res$PPA = res$PPA.x *res$PPA.y
res = res[order(res$PPA, decreasing=T),]

*Choose a cutoff of  PPA, then re-adjust the PPA by multipling by 1 if the Locus is known

In [4]:
seg = res[res$PPA >0.3, ]

In [5]:
gwas = "/publicdata/gwas_summary_stats_20180124/vanSetten_2018/PR_interval_July2018_summary_results.hg19.txt"
p = paste( "grep -E '", paste(seg$id, collapse="|"), "' ", gwas ," > PR_interval_selected_variants.tsv", sep="")

In [6]:
system(p)

In [7]:
r = read.table("PR_interval_selected_variants.tsv")

In [8]:
seg = merge(seg, r[,c('V2', 'V10')], by.x="id", by.y="V2")
colnames(seg)[ncol(seg)] = "P_value"


In [9]:
writeLines(as.character(seg$id), "TOP_rs_variants_PR")

In [10]:
system("module load cardips")

command1 = "vcftools --vcf /publicdata/haploreg_v4.0_20151021/haploreg_v4.0_20151021.vcf --snps TOP_rs_variants_PR --recode --recode-INFO-all --out PR_interval"
### double escape to use in R
command2 = paste("bcftools query -f ", "'%ID\\t%INFO/DBSNP\\t%INFO/NEAREST_GENCODE\\n' ",
                    "PR_interval.recode.vcf > PR_haploreg.tab", sep="")
           
system(command1)
system(command2)

In [11]:
a = read.table("PR_haploreg.tab")

colnames(a)=c('id', 'dbSNP', 'gencode')

seg = merge(seg, a, by=1)

In [12]:
seg = seg[order(seg$PPA, decreasing=T),]
write.csv(seg, 'fgwas_results_PR_interval.csv', row.names=FALSE)

In [13]:
seg = subset(seg, dbSNP!="NSM")

In [14]:
se = seg[,c('NKX25_ASE','NKX25','H3K27AC','ATAC')]

In [15]:
mat = seg[rowSums(se)>0 & seg$P_value<1e-3 , c('id',  'PPA','NKX25_ASE','NKX25','H3K27AC','ATAC')]

In [16]:
mat$GWAS_P = mat$id %in% seg$id[seg$P_value<1e-8]
#mat$GWAS_other      = mat$id %in% seg$id[seg$P_value<1e-4]

mat2 = mat[,2:length(mat)]
rownames(mat2)=mat$id

lab = mat2
lab[, 2:length(mat2)]<-NA
lab[,1]<-round(lab[,1],2)

suppressPackageStartupMessages(library(RColorBrewer))
cols = brewer.pal(9, "Reds")[2:8]

breaks=seq(0.5, 1.1, by=0.1) # cutoff 0.5
breaks=seq(0.3, 1.1, by=0.1) # cutoff 0.3

my_palette = c(colorRampPalette(cols)(length(breaks)-2), "orange")

In [17]:
mat2[mat2==0]<- NA
mat2[mat2==TRUE]<- 1
mat2[mat2==1]<- 2

In [ ]:
#pdf('Table_snps_NKX_PR_new.pdf')
heatmap.2(as.matrix(mat2[1:5,]), 
         Colv=F,Rowv=F, srtRow=0, srtCol=45, offsetRow=-1, offsetCol=-0.5,dendrogram="none",
          keysize=1, margins =c(30,26), trace="none",key.title="OR", tracecol=NA,
          cexRow=1, cexCol=1, na.color="white", col=my_palette, breaks=breaks, cellnote=lab, notecex=0.8)
#dev.off()

### Forest plot of the parametes aenriched

In [23]:
params= data.frame()
for (tr in  c("HR", 'A_fib','PR_interval' )){
par = read.table(paste("../fgwas_analysis/", tr, "/pipeline_out/Final_Model/Final_Model.params", sep=""), header=T, 
                          stringsAsFactor=F, sep=" ") 
 par$trait = tr   
par = par[-1,]
params =  rbind(params, par)
    }
                    


In [24]:
params$parameter = substr(params$parameter, 1, nchar(params$parameter)-3)

In [25]:
params[4, 2:3] <- -6
params[10, 2:3] <- -6
params[15, 2] <- -6

In [26]:
params$CI_lo = as.numeric(params$CI_lo)

In [27]:
#params = params[order(params$parameter),]

In [28]:
#params = params[order(params$trait),]

In [31]:
save =params

In [249]:
options(warn=-1)
pdf('/frazer01/home/paola/Family1070/private_output/Fgwas/Enrichment_fgwas_final_model.pdf')
colors=c(rep(c("orange","gray20", "deepskyblue3"),5))
colors=c(rep(c("deepskyblue3","gray30", "orange"), each=5))

par(mfrow=c(2,2), pin=c(1.5,2), las=1)
plotCI(x=params$estimate, y=15:1, ui=params$CI_hi, li=params$CI_lo, err="x", xlim=c(-6,+6),
        col=colors, pch=c(19,1)[(params$CI_lo<0)+1], cex=1.2, lwd=2,
       sfrac=0,gap=0,  add=FALSE, xlab="fgwas ln(Enrichment)" , ylab="", axes=F)
axis(side=2, at=15:1, label=params$parameter)         
axis(side=1, at=-6:6 , label=-6:6)
box()
grid()
abline(v=0, lwd=1, lty=2, col="red")
options(warn=0)
dev.off()

png 
  2

In [44]:
options(warn=-1)
pdf('/frazer01/home/paola/Family1070/private_output/Fgwas/Enrichment_fgwas_heart_rate.pdf')
colors=c(rep(c("deepskyblue3","gray30", "orange"), each=5))
params = save[1:5,]
par(mfrow=c(2,2), pin=c(1.5,1), las=1)
plotCI(x=params$estimate, y=5:1, ui=params$CI_hi, li=params$CI_lo, err="x", xlim=c(-6,+6),
        col=colors, pch=c(19,1)[(params$CI_lo<0)+1], cex=1.2, lwd=2,
       sfrac=0,gap=0,  add=FALSE, xlab="fgwas ln(Enrichment)" , ylab="", axes=F)
axis(side=2, at=5:1, label=params$parameter)         
axis(side=1, at=-6:6 , label=-6:6)
box()
grid()
abline(v=0, lwd=1, lty=2, col="red")
options(warn=0)
dev.off()

png 
  2

In [43]:
options(warn=-1)
colors=c(rep(c("gray30", "orange"), each=5))
pdf('/frazer01/home/paola/Family1070/private_output/Fgwas/Enrichment_fgwas_AF_PR.pdf')

params = save[6:15,]
par(mfrow=c(2,2), pin=c(1.5,1.5), las=1)
plotCI(x=params$estimate, y=10:1, ui=params$CI_hi, li=params$CI_lo, err="x", xlim=c(-6,+6),
        col=colors, pch=c(19,1)[(params$CI_lo<0)+1], cex=1.2, lwd=2,
       sfrac=0,gap=0,  add=FALSE, xlab="fgwas ln(Enrichment)" , ylab="", axes=F)
axis(side=2, at=10:1, label=params$parameter)         
axis(side=1, at=-6:6 , label=-6:6)
box()
grid()
abline(v=0, lwd=1, lty=2, col="red")
options(warn=0)
dev.off()

png 
  2

In [233]:
get_single_enrichment = function(dir) {

files = list.files(paste(dir,"pipeline_out/Buildup_Model/First_Run",sep="/"))
files = files[grepl(pattern="params", x=files)]
for (file in files){
solo = read.table(paste(dir,"pipeline_out/Buildup_Model/First_Run",sep="/", file),  
                  header=T, stringsAsFactors=F)
if(file == files[1]){
    combo = solo
} else {
    combo = rbind(combo, solo)   
    }
    }
return(combo)
}

In [234]:
singpar=data.frame()
for (tr in  c("HR", 'A_fib','PR_interval' )){
dir = paste("../fgwas_analysis/", tr, sep="")
par = get_single_enrichment(dir= dir)
par = par[par$parameter!="pi_region",]
par$trait = tr  
singpar =  rbind(singpar, par)
    }

In [235]:
singpar[2, 2:3] <- -6
singpar[7, 2:3] <- -6
singpar[12, 2] <- -6

In [236]:
singpar$CI_lo = as.numeric(singpar$CI_lo)

In [242]:
singpar = singpar[with(singpar, order(trait, -estimate)),]

In [250]:
options(warn=-1)
pdf('/frazer01/home/paola/Family1070/private_output/Fgwas/Enrichment_fgwas_ss_model.pdf')

colors=c(rep(c("gray30","deepskyblue3", "orange"), each=5))

par(mfrow=c(2,2), pin=c(1.5,2), las=1)
plotCI(x=singpar$estimate, y=15:1, ui=singpar$CI_hi, li=singpar$CI_lo, err="x", xlim=c(-6,+7),
        col=colors, pch=c(19,1)[(singpar$CI_lo<0)+1], cex=1.2, lwd=2,
       sfrac=0,gap=0,  add=FALSE, xlab="fgwas ln(Enrichment)" , ylab="", axes=F)
axis(side=2, at=15:1, label=singpar$parameter)         
axis(side=1, at=-6:7 , label=-6:7)
box()
grid()
abline(v=0, lwd=1, lty=2, col="red")
options(warn=0)
dev.off()

png 
  2